In [11]:
import pandas as pd
import os
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment, PatternFill

# Example mapping of course codes to program names
COURSE_CODE_TO_PROGRAM = {
    "ECO102": "Economics",
    "CHM102": "Chemistry",
    "PHY108": "Physics",
    # Add more mappings as needed...
}

def extract_level(course_code):
    """Extract the level from the course code."""
    if pd.isna(course_code):
        return None
    if "/" in course_code:  # For combined courses
        course_code = course_code.split("/")[0]
    digits = ''.join(filter(str.isdigit, course_code))
    if digits and len(digits) >= 3:
        return int(digits[0])  # The first digit indicates the level
    return None

def process_courses(sheet_path):
    """Process the 'Second Semester' sheet to extract and organize course data."""
    df = pd.read_excel(sheet_path, sheet_name="Second Semester")
    courses = []

    # Replace NaN with empty strings to avoid errors
    df = df.fillna({"COURSE": "", "PROGRAMMES OFFERING": "", "TYPE": ""})

    # Dictionary to keep track of student numbers for each program
    program_student_counts = {}

    for _, row in df.iterrows():
        course_code = str(row["COURSE"]).strip()
        programs = str(row["PROGRAMMES OFFERING"]).strip()
        exam_type = str(row["TYPE"]).strip()
        level = extract_level(course_code)

        # Split programs into a list of individual programs or leave combined programs intact
        if "," in programs:
            program_list = [program.strip() for program in programs.split(",")]
            is_combined = True
        else:
            program_list = [programs.strip()]
            is_combined = False

        # Generate student counts for each program
        for program in program_list:
            if program not in program_student_counts:
                program_student_counts[program] = 0
            program_student_counts[program] += 30  # Example: 30 students per program

        # For combined courses, aggregate student counts and program names
        if is_combined:
            combined_course_name = "Combined: " + ", ".join(program_list)
            combined_student_count = sum(program_student_counts[program] for program in program_list)
            courses.append({
                "Course Code": course_code,
                "Program": combined_course_name,
                "Exam Type": exam_type,
                "Level": level,
                "Number of Students": combined_student_count
            })
        else:
            # For individual programs, add directly
            for program in program_list:
                courses.append({
                    "Course Code": course_code,
                    "Program": program,
                    "Exam Type": exam_type,
                    "Level": level,
                    "Number of Students": program_student_counts[program]
                })

    return pd.DataFrame(courses)


def process_venues(sheet_path):
    """Process the 'Venues' sheet."""
    df = pd.read_excel(sheet_path, sheet_name="Venues")
    return df.rename(columns={"HALL": "Venue", "CODE": "Code", "EXAM CAPACITY": "Capacity"})

def process_staff(sheet_path):
    """Process the 'List of Staff' sheet."""
    df = pd.read_excel(sheet_path, sheet_name="List of Staff")
    return df.rename(columns={"NAME": "Name", "COLLEGE": "College", "DEPARTMENT": "Department", "PHONE NUMBER": "Phone"})

def process_cbt_venues():
    """Generate the CBT venues data."""
    cbt_venues = [
        {"Venue": "Cafe", "Code": "C1", "Capacity": 100},
        {"Venue": "Lab 1", "Code": "L1", "Capacity": 50},
        {"Venue": "Lab 2", "Code": "L2", "Capacity": 50},
        {"Venue": "Lab 3", "Code": "L3", "Capacity": 50},
        {"Venue": "Lab 4", "Code": "L4", "Capacity": 50},
    ]
    return pd.DataFrame(cbt_venues)

def adjust_column_width_and_style(writer, sheet_name):
    """Adjust column width and apply header style in Excel."""
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]

    for column_cells in worksheet.columns:
        max_length = max((len(str(cell.value)) for cell in column_cells if cell.value), default=0)
        adjusted_width = max_length + 4
        worksheet.column_dimensions[column_cells[0].column_letter].width = adjusted_width

    header_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
    header_font = Font(bold=True)
    header_alignment = Alignment(horizontal="center", vertical="center")

    for cell in worksheet[1]:
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = header_alignment

def main(input_file, output_file):
    try:
        courses_df = process_courses(input_file)
        venues_df = process_venues(input_file)
        staff_df = process_staff(input_file)
        cbt_venues_df = process_cbt_venues()

        with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
            courses_df.to_excel(writer, sheet_name="Courses", index=False)
            adjust_column_width_and_style(writer, "Courses")

            venues_df.to_excel(writer, sheet_name="Venues", index=False)
            adjust_column_width_and_style(writer, "Venues")

            staff_df.to_excel(writer, sheet_name="Staff", index=False)
            adjust_column_width_and_style(writer, "Staff")

            cbt_venues_df.to_excel(writer, sheet_name="CBT Venues", index=False)
            adjust_column_width_and_style(writer, "CBT Venues")

        print(f"Processed data saved to {output_file}")
    except Exception as e:
        print(f"Error processing the data: {e}")

if __name__ == "__main__":
    input_excel = "Data for Exam timetable.xlsx"
    output_excel = "datam10.xlsx"

    if os.path.exists(input_excel):
        main(input_excel, output_excel)
    else:
        print(f"Input file '{input_excel}' not found. Please provide the correct file.")


Processed data saved to datam10.xlsx


In [26]:
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill
from datetime import datetime, timedelta
import pandas as pd

def allocate_courses(courses_df, venues_df, cbt_venues_df):
    """Allocate courses to venues and time slots while adhering to constraints."""
    # Define time slots
    time_slots = [
        {"Session": "Morning", "Start": "08:00 AM", "End": "10:30 AM"},
        {"Session": "Afternoon", "Start": "11:00 AM", "End": "01:00 PM"},
        {"Session": "Evening", "Start": "02:00 PM", "End": "04:30 PM"}
    ]

    # Define two weeks of dates (Monday-Friday + Saturday overflow)
    start_date = datetime.strptime("2024-04-01", "%Y-%m-%d")  # Adjust as needed
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    dates = []
    for i in range(14):  # Two weeks
        current_date = start_date + timedelta(days=i)
        if current_date.strftime("%A") in days:
            dates.append(current_date)

    # Prepare data structures for scheduling
    cbt_allocated = []
    written_allocated = []
    unscheduled = []
    invigilator_counter = 1  # To assign unique invigilator IDs

    # Venue availability
    venue_availability = {
        venue: {date: {slot["Session"]: {"Used": False, "Capacity": cap} for slot in time_slots}
                for date in dates}
        for venue, cap in zip(venues_df["Venue"], venues_df["Capacity"])
    }
    cbt_venue_availability = {
        venue: {date: {slot["Session"]: {"Used": False, "Capacity": cap} for slot in time_slots}
                for date in dates}
        for venue, cap in zip(cbt_venues_df["Venue"], cbt_venues_df["Capacity"])
    }

    # Schedule courses
    for _, course in courses_df.iterrows():
        scheduled = False
        student_count = course["Number of Students"]
        exam_type = str(course["Exam Type"]).strip().lower()
        course_code = course["Course Code"]

        # Choose venues based on exam type
        venues_to_check = cbt_venue_availability if exam_type == "cbt" else venue_availability

        # Loop through dates and sessions to schedule exams
        for date in dates:
            day_name = date.strftime("%A")
            for slot in time_slots:
                suitable_venues = [
                    venue for venue, slots in venues_to_check.items()
                    if not slots[date][slot["Session"]]["Used"]
                ]

                # Check if combined capacity can accommodate students
                if suitable_venues:
                    total_capacity = sum(venues_to_check[venue][date][slot["Session"]]["Capacity"]
                                         for venue in suitable_venues)
                    if student_count <= total_capacity:
                        # Mark venues as used
                        for venue in suitable_venues:
                            venues_to_check[venue][date][slot["Session"]]["Used"] = True

                        if exam_type == "cbt":
                            invigilators = [
                                f"C1-{invigilator_counter}", f"L1-{invigilator_counter+1}",
                                f"L2-{invigilator_counter+2}", f"L3-{invigilator_counter+3}",
                                f"L4-{invigilator_counter+4}", f"E-{invigilator_counter+5}"
                            ]
                            invigilator_counter += 6
                            cbt_allocated.append({
                                "Date": date.strftime("%Y-%m-%d"),
                                "Day": day_name,
                                "Time": f"{slot['Start']} - {slot['End']}",
                                "Course": course_code,
                                "Programme": course["Program"],
                                "Invigilators": ", ".join(invigilators)
                            })
                        else:
                            invigilators = max(2, (student_count // 40) + 1)
                            written_allocated.append({
                                "Date": date.strftime("%Y-%m-%d"),
                                "Day": day_name,
                                "Session": slot["Session"],
                                "Start": slot["Start"],
                                "End": slot["End"],
                                "Course Code": course_code,
                                "Course Name": course["Program"],
                                "Venue": ", ".join(suitable_venues),
                                "Invigilators": invigilators,
                                "Number of Students": student_count
                            })
                        scheduled = True
                        break
            if scheduled:
                break

        if not scheduled:
            # Track unscheduled courses
            unscheduled.append({
                "Course Code": course_code,
                "Program": course["Program"],
                "Exam Type": exam_type.capitalize(),
                "Number of Students": student_count,
                "Reason": "No available time slots or venue capacity"
            })

    return pd.DataFrame(cbt_allocated), pd.DataFrame(written_allocated), pd.DataFrame(unscheduled)

def write_timetable(output_file, cbt_df, written_df, unscheduled_df):
    """Write the allocated and unscheduled courses to an Excel file."""
    workbook = Workbook()

    # CBT Exams Sheet
    cbt_sheet = workbook.active
    cbt_sheet.title = "CBT Exams"
    cbt_headers = ["Date", "Day", "Time", "Course", "Programme", "Invigilators"]
    cbt_sheet.append(cbt_headers)
    for _, row in cbt_df.iterrows():
        cbt_sheet.append(list(row))

    # Written Exams Sheet
    written_sheet = workbook.create_sheet("Written Exams")
    written_headers = ["Date", "Day", "Session", "Start", "End", "Course Code", "Course Name",
                       "Venue", "Invigilators", "Number of Students"]
    written_sheet.append(written_headers)
    for _, row in written_df.iterrows():
        written_sheet.append(list(row))

    # Apply text wrapping and adjust column widths
    for sheet in [cbt_sheet, written_sheet]:
        for row in sheet.iter_rows():
            for cell in row:
                cell.alignment = Alignment(wrap_text=True)
        for col in sheet.columns:
            max_length = max((len(str(cell.value)) for cell in col if cell.value), default=0)
            adjusted_width = min(40, max(10, max_length + 2))
            sheet.column_dimensions[col[0].column_letter].width = adjusted_width

    # Unscheduled Courses Sheet
    unscheduled_sheet = workbook.create_sheet("Unscheduled Courses")
    unscheduled_headers = ["Course Code", "Program", "Exam Type", "Number of Students", "Reason"]
    unscheduled_sheet.append(unscheduled_headers)
    for _, row in unscheduled_df.iterrows():
        unscheduled_sheet.append(list(row))

    # Save workbook
    workbook.save(output_file)
    print(f"Timetable saved to {output_file}")

def main(input_file, output_file):
    # Load the processed data
    courses_df = pd.read_excel(input_file, sheet_name="Courses")
    venues_df = pd.read_excel(input_file, sheet_name="Venues")
    cbt_venues_df = pd.read_excel(input_file, sheet_name="CBT Venues")

    # Allocate courses to slots and venues
    cbt_df, written_df, unscheduled_df = allocate_courses(courses_df, venues_df, cbt_venues_df)

    # Write the results to the output file
    write_timetable(output_file, cbt_df, written_df, unscheduled_df)

if __name__ == "__main__":
    input_excel = "datam10.xlsx"  # Input file from Script 1
    output_excel = "timetablem105.xlsx"  # Output timetable file
    main(input_excel, output_excel)

Timetable saved to timetablem105.xlsx
